In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import re
import os

from collections import Counter
from nlppln.utils import get_files
from adhtools.split_text_openiti_markers import smart_strip

in_dir = '/home/jvdzwaan/data/adh-corpora/fiqh_corpus/txt/'

regex = r'\#\#\# (?P<level>\|+) (?P<header>.+?)\n|' \
            '@(?P<source>[QH])B@(?P<quote>.+?)@(?P=source)E@'

max_level = 0
lvls = []
qs = []

result = {}

for in_file in get_files(in_dir):
    doc_id = os.path.splitext(os.path.basename(in_file))[0]
    
    levels = []
    quotes = []
    c = Counter()
    with open(in_file) as f:
        text = f.read()
        
    for m in re.finditer(regex, text):
        matches = m.groupdict()
        
        if matches['header']:
            title = smart_strip(matches.get('header', ''))
            level = len(matches.get('level'))
                                
            if level > max_level:
                max_level = level
        
            levels.append(level)
            lvls.append(level)
            c[level] += 1
        else:
            source = matches.get('source')
            quotes.append(source)
            qs.append(source)
            c[source] += 1
            
    result[doc_id] = {'header_1': c[1],
                      'header_2': c[2],
                      'header_3': c[3],
                      'quran': c['Q'],
                      'hadith': c['H']}        
        
    #print(in_file, set(levels))
    #print('number of header markers:', sum(c.values()))
    #print(set(quotes))

print(max_level)
print(set(lvls))
print(set(qs))
    
new_metadata = pd.DataFrame.from_dict(result).T
new_metadata.index.names = ['BookURI']
new_metadata

In [ ]:
# merge with old metadata

md_file = '/home/jvdzwaan/data/adh-corpora/fiqh_corpus/Meta/Metadata_Fiqh.csv'

md = pd.read_csv(md_file, index_col=1)
md = md.drop(columns=['header_1', 'header_2', 'header_3', 'quran', 'hadith'])
md

In [ ]:
r = md.join(new_metadata)

In [ ]:
# remove bookuri as index and turn it into a column
r = r.reset_index()

In [ ]:
r.head()

In [ ]:
r.columns

In [ ]:
# reorder the columns
cols = ['Order', 'BookURI', 'BookSource', 'BookTITLE_SHORT', 'BookTITLE',  
       'BookSUBJ', 'Number_of_tokens', 'AuthorAKA', 'AuthorNAME',
       'AuthorBORNH', 'AuthorBORNC', 'AuthorDIEDH', 'AuthorDIEDC', 'Century',
       'All_tokens_per-century', 'Author_Geographical_Area', #'Tagging',
       'BookVOLS', 'header_1', 'header_2', 'header_3', 'quran', 'hadith']
r = r[cols]
r.head()

In [ ]:
# remove tagging column (it is no longer up to date)
r = r.drop('Tagging', axis=1)

In [ ]:
r.to_csv(md_file, index=False)
r

In [ ]:
md.to_csv(md_file, index=False)